#### Experimento quedandome solamente con las columnas numericas.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from math import sqrt

In [2]:
##METRICA

def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [21]:
df = pd.read_csv("train.csv")

In [22]:
df.shape

(240000, 23)

In [23]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [25]:
df.fecha.head()

0    2015-08-23 00:00:00
1    2013-06-28 00:00:00
2    2015-10-17 00:00:00
3    2012-03-09 00:00:00
4    2016-06-07 00:00:00
Name: fecha, dtype: object

In [26]:
df.fecha = pd.to_datetime(df['fecha'])

In [27]:
df.fecha.head()

0   2015-08-23
1   2013-06-28
2   2015-10-17
3   2012-03-09
4   2016-06-07
Name: fecha, dtype: datetime64[ns]

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [29]:
df['year'] = df.fecha.dt.year
df['mes'] = df.fecha.dt.month
df['dia_del_anio'] = df.fecha.dt.day

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 26 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [31]:
columnsDrop = ['id','titulo','descripcion','tipodepropiedad','direccion','ciudad','provincia','idzona','lat','lng','fecha']

In [32]:
X = df.drop(['precio'] + columnsDrop,axis = 1)
y = df['precio']

In [33]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 14 columns):
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240000 non-null float64
centroscomercialescercanos    240000 non-null float64
year                          240000 non-null int64
mes                           240000 non-null int64
dia_del_anio                  240000 non-null int64
dtypes: float64(11), int64(3)
memory usage: 25.6 MB


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [35]:
params_baseline_xgb = {'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 0.8}

In [36]:
model = XGBRegressor(**params_baseline_xgb)

In [37]:
%%time
results = model.fit(X_train,y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:01:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 49.5 s, sys: 33.4 ms, total: 49.5 s
Wall time: 12.4 s


In [38]:
results

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [39]:
pred = model.predict(X_test)

In [40]:
RMSLE(y_test, pred)

0.4871370099600868

In [43]:
param_dist = {'n_estimators':[50, 100],
              'max_depth':[5,10],              
              'subsample': [0.5,0.8,1],
              'colsample_bytree': [0.5,0.8,1],              
              'learning_rate':[0.01, 0.1, 0.3]
              }

In [47]:
random_search_xgb = RandomizedSearchCV(model,
                                       param_distributions = param_dist,
                                       n_iter = 20,
                                       cv = 5,
                                       verbose=5)

In [48]:
%%time
random_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] subsample=0.5, n_estimators=50, max_depth=10, learning_rate=0.1, colsample_bytree=0.8 
[18:08:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  subsample=0.5, n_estimators=50, max_depth=10, learning_rate=0.1, colsample_bytree=0.8, score=0.569, total=   5.7s
[CV] subsample=0.5, n_estimators=50, max_depth=10, learning_rate=0.1, colsample_bytree=0.8 
[18:08:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s
/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.5, n_estimators=50, max_depth=10, learning_rate=0.1, colsample_bytree=0.8, score=0.561, total=   5.9s
[CV] subsample=0.5, n_estimators=50, max_depth=10, learning_rate=0.1, colsample_bytree=0.8 
[18:08:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.5, n_estimators=100, max_depth=5, learning_rate=0.1, colsample_bytree=0.5, score=0.552, total=   4.8s
[18:20:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 11.4min finished
/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 42min 44s, sys: 3min 45s, total: 46min 29s
Wall time: 11min 38s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.8, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=10, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=4,
                                          objective='reg:linear',
                                          random_st...
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=0.8,
                                

In [49]:
params_opt_xgb = random_search_xgb.best_params_

In [50]:
params_opt_xgb

{'subsample': 1,
 'n_estimators': 100,
 'max_depth': 10,
 'learning_rate': 0.1,
 'colsample_bytree': 0.8}

In [51]:
params_baseline_xgb2 = params_baseline_xgb.copy()

In [52]:
params_baseline_xgb2

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 0.8}

In [53]:
params_baseline_xgb2.update(params_opt_xgb)

In [54]:
params_baseline_xgb2

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 1}

In [56]:
modelOpt = XGBRegressor(**params_baseline_xgb2)

In [58]:
%%time
resultsOpt = modelOpt.fit(X_train,y_train)

[18:51:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 50.8 s, sys: 4.03 s, total: 54.8 s
Wall time: 13.8 s


In [61]:
predOpt = modelOpt.predict(X_test)

In [62]:
RMSLE(y_test, predOpt) #TODO TONEADO Y OPTIMIZADO

0.48631670272787353

In [63]:
RMSLE(y_test, pred) #Muy poca mejora al respecto con todo los pasos anteriores.

0.4871370099600868